# Time comparison between pywake and my model simulations

In [3]:
import utils.utils as utils
import xarray as xr
import time
import itertools
import torch
import torch.nn as nn
import numpy as np

from py_wake.deficit_models import EddyViscosityModel

import utils.utils as utils
import utils.pywake_utils as py_wake_utils
import utils.data_utils as data_utils

# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)

# default parameters
TURBINE_X = [0]
TURBINE_Y = [0]
WIND_DIRECTION = 270 

# IEA37 values
TURBINE_DIAMETER = 198
TURBINE_HUB_HEIGHT = 119
TURBINE_POWER_NORM = 10000

# discretization factors
X_START_FACTOR = 2
X_END_FACTOR = 20
Y_START_FACTOR = -5
Y_END_FACTOR = 5
GRID_STEP_FACTOR = 1/8

# parameters for data generation
WS_RANGE = range(3, 26)
# TODO change the step sizes?
TI_STEP = 0.01
CT_STEP = 0.01
TIs = utils.my_arange(0, 1, TI_STEP)
CTs = utils.my_arange(0.2, 24/25, CT_STEP)

In [4]:
horizontal_grid = py_wake_utils.get_discretized_grid(TURBINE_DIAMETER,
                                       X_START_FACTOR, X_END_FACTOR, Y_START_FACTOR, Y_END_FACTOR,
                                       GRID_STEP_FACTOR)

ws_to_list = dict()
for wind_speed in WS_RANGE:
    datasets = list()

    start_time = time.time()

    for ti, ct in itertools.product(TIs, CTs):
        site = py_wake_utils.get_site(ti=ti, ws=wind_speed)
        wind_turbine = py_wake_utils.get_wind_turbine(TURBINE_DIAMETER, TURBINE_HUB_HEIGHT, TURBINE_POWER_NORM,
                                        constant_ct=ct, ti=ti)
        
        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        ds = py_wake_utils.generate_wake_dataset(ainslie_model, wind_speed, WIND_DIRECTION,
                                    TURBINE_DIAMETER, TURBINE_X, TURBINE_Y,
                                    horizontal_grid, wind_turbine)
        datasets.append(ds)

    filepath = data_utils.get_filepath(X_START_FACTOR, X_END_FACTOR, Y_START_FACTOR, Y_END_FACTOR,
                                  GRID_STEP_FACTOR, wind_speed, TI_STEP, CT_STEP)
    final_ds = xr.concat([d.stack(z=['x:D', 'y:D', 'ti', 'ct']) for d in datasets], 'z').unstack('z')
    cacca = final_ds.load()
    
    end_time = time.time()
    execution_time = end_time - start_time
    num_simulations = len(datasets)
    print(f"For {wind_speed=}] Execution time: {execution_time} seconds for {num_simulations} simulations -> {(execution_time/num_simulations):2f}s per simulation")
    ws_to_list[wind_speed] = [num_simulations, execution_time]

tot_simulations = sum([num_sim for num_sim, _ in ws_to_list.values()])
tot_time = sum([ex_time for _, ex_time in ws_to_list.values()])
print(f"\nOverall execution time for {tot_simulations} simulations: {tot_time} seconds -> {tot_time/tot_simulations:2f}s per simulation")


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


For wind_speed=3] Execution time: 8515.641827106476 seconds for 7600 simulations -> 1.120479s per simulation


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


For wind_speed=4] Execution time: 8498.752365112305 seconds for 7600 simulations -> 1.118257s per simulation


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


For wind_speed=5] Execution time: 8514.600550889969 seconds for 7600 simulations -> 1.120342s per simulation


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


For wind_speed=6] Execution time: 8463.694307804108 seconds for 7600 simulations -> 1.113644s per simulation


KeyboardInterrupt: 

In [8]:
MODEL_PATH = "saved_models/discr_factors_x2_20_y-5_5_step0.125_TIstep0.01_CTstep0.01/multivariate_NN_layers50-500-2500.pt"
HIDDEN_LAYERS_UNITS = [int(lu) for lu in MODEL_PATH.split("layers")[-1].split(".pt")[0].split("-")]

class MultivariateNN(nn.Module):
    def __init__(self, input_space, output_space) -> None:
        super(MultivariateNN, self).__init__()
        layer_units = [input_space] + HIDDEN_LAYERS_UNITS + [output_space]
        layers = list()
        for first, second in zip(layer_units, layer_units[1:]):
            layers += [nn.Linear(first, second), nn.ReLU()]
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

model = MultivariateNN(2, 11520)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

ws_to_list = dict()
with torch.no_grad():
    for wind_speed in WS_RANGE:
        datasets = list()

        start_time = time.time()

        for ti, ct in itertools.product(TIs, CTs):
            wake_field = model(torch.FloatTensor([ti, ct]))
            datasets.append(wake_field)

        end_time = time.time()
        execution_time = end_time - start_time
        num_simulations = len(datasets)
        print(f"For {wind_speed=}] Execution time: {execution_time} seconds for {num_simulations} simulations -> {(execution_time/num_simulations):2f}s per simulation")
        ws_to_list[wind_speed] = [num_simulations, execution_time]

tot_simulations = sum([num_sim for num_sim, _ in ws_to_list.values()])
tot_time = sum([ex_time for _, ex_time in ws_to_list.values()])
print(f"\nOverall execution time for {tot_simulations} simulations: {tot_time} seconds -> {tot_time/tot_simulations:2f}s per simulation")

For wind_speed=3] Execution time: 28.539746046066284 seconds for 7600 simulations -> 0.003755s per simulation
For wind_speed=4] Execution time: 28.614805936813354 seconds for 7600 simulations -> 0.003765s per simulation
For wind_speed=5] Execution time: 29.45336079597473 seconds for 7600 simulations -> 0.003875s per simulation
For wind_speed=6] Execution time: 28.728474140167236 seconds for 7600 simulations -> 0.003780s per simulation
For wind_speed=7] Execution time: 29.862894773483276 seconds for 7600 simulations -> 0.003929s per simulation
For wind_speed=8] Execution time: 29.59728980064392 seconds for 7600 simulations -> 0.003894s per simulation
For wind_speed=9] Execution time: 29.345993041992188 seconds for 7600 simulations -> 0.003861s per simulation
For wind_speed=10] Execution time: 29.615293979644775 seconds for 7600 simulations -> 0.003897s per simulation
For wind_speed=11] Execution time: 29.477768898010254 seconds for 7600 simulations -> 0.003879s per simulation
For wind_s

# Input parameters tuning

In [17]:
import random
import utils.plotting

DEFICIT_THRESHOLD = 1/100 #putting it to 1/1000, the region becomes too big

X_START_FACTOR = 2
X_END_FACTOR = 50
Y_START_FACTOR = -10
Y_END_FACTOR = 10

horizontal_grid = py_wake_utils.get_discretized_grid(TURBINE_DIAMETER,
                                       X_START_FACTOR, X_END_FACTOR, Y_START_FACTOR, Y_END_FACTOR,
                                       GRID_STEP_FACTOR)

x_ranges = list()
y_ranges = list()

for wind_speed in random.sample(list(WS_RANGE), 5):
    for ti, ct in random.sample(sorted(itertools.product(TIs, CTs)), 200):
        #print(f"{wind_speed=}\t{ti=}\t{ct=}")
        site = py_wake_utils.get_site(ti=ti, ws=wind_speed)
        wind_turbine = py_wake_utils.get_wind_turbine(TURBINE_DIAMETER, TURBINE_HUB_HEIGHT, TURBINE_POWER_NORM,
                                        constant_ct=ct, ti=ti)
        
        # single wake model
        ainslie_model = EddyViscosityModel(site, wind_turbine)

        df = py_wake_utils.generate_wake_dataset(ainslie_model, wind_speed, WIND_DIRECTION,
                                    TURBINE_DIAMETER, TURBINE_X, TURBINE_Y,
                                    horizontal_grid, wind_turbine)\
                .to_dataframe()\
                .reset_index()\
                .rename(columns={'x:D': 'x/D', 'y:D': 'y/D'})\
                .sort_values(by=['x/D', 'y/D'])

        filtered_df = df[df['wind_deficit'] > DEFICIT_THRESHOLD]
        if filtered_df.empty:
            utils.plotting.plot_deficit_map(df)
            continue

        x_range = np.nanmin(filtered_df['x/D']), np.nanmax(filtered_df['x/D'])
        y_range = np.nanmin(filtered_df['y/D']), np.nanmax(filtered_df['y/D'])

        #print("Range of x/D:", x_range)
        #print("Range of y/D:", y_range)
        x_ranges.append(x_range)
        y_ranges.append(y_range)

In [18]:
import statistics
x_ends = sorted([x_range[1] for x_range in x_ranges])
print(f"For x-end, min is: {min(x_ends)} and max is: {max(x_ends)},"+\
      f" with an average of {sum(x_ends) / len(x_ends)} and a median of {statistics.median(x_ends)}")

y_starts = sorted([y_range[0] for y_range in y_ranges])
print(f"For y-start, min is: {min(y_starts)} and max is: {max(y_starts)},"+\
      f" with an average of {sum(y_starts) / len(y_starts)} and a median of {statistics.median(y_starts)}")
y_ends = sorted([y_range[1] for y_range in y_ranges])
print(f"For y-end, min is: {min(y_ends)} and max is: {max(y_ends)},"+\
      f" with an average of {sum(y_ends) / len(y_ends)} and a median of {statistics.median(y_ends)}")

For x-end, min is: 6.25 and max is: 49.875, with an average of 30.989 and a median of 30.625
For y-start, min is: -2.0 and max is: -0.75, with an average of -1.535875 and a median of -1.625
For y-end, min is: 0.75 and max is: 2.0, with an average of 1.535875 and a median of 1.625
